# Imports

In [14]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as Layer
import numpy as np
import keras_tuner as kt
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
# from DatasetFormatting import *
import statistics


In [15]:
import re
import pandas as pd

movieFields = ['id', 'title', 'vote_average', 'vote_count', 'genres']
castFields = ['id', 'cast']

movieData = pd.read_csv('archive/movies_metadata.csv', skipinitialspace=True, usecols=movieFields, engine='python')
castData = pd.read_csv('archive/credits.csv', skipinitialspace=True, usecols=castFields, engine='python')

dfMovies = pd.DataFrame(movieData)
dfCast = pd.DataFrame(castData)

df = pd.merge(dfMovies, dfCast, how='inner')


# ---------------------------------------------------------------------------- #
# The following section formats the cast and creates a clean list of actor names.

unformattedNames = df['cast']

# main list of cast grouped by movies
cast_master_copy = []

# list of overall cast by individual names
actor_name_list = []

# reads through the cast string for each movie
for unformattedNameString in unformattedNames:

    # creates new list for current movie
    grouped = ""

    # pulls actors' names out of the paragraph
    line_list = re.findall("(?<=\'name\': )(.*?)(?=,)", unformattedNameString)
    
    # converts list of cast for one movie into a string to manipulate further 
    line_string = str(line_list)

    # separates cast into a unique list per movie
    line_split = line_string.split(", ")

    actor_count = 0
    # reads through each actor per movie
    for name in line_split:
        if (actor_count == 5):
            break
     
        # removes extraneous symbols from the actors' names
        name = name.replace("\"", "")
        name = name.replace("\'", "")
        name = name.replace("[", "")
        name = name.replace("]", "")
        name = name.replace(" ", "")

        # adds actors to two working lists
        # grouped: actors are in lists by movies
        # actor_name_list: puts every actors in one list
        grouped += name + ' '
        actor_name_list.append(name)
        
        actor_count += 1

    # adds formatted cast members to list
    grouped = grouped[:-1]
    cast_master_copy.append(grouped)

# assigns formatted cast to 'cast' column of DataFrame
df['cast'] = cast_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section formats the genre and adds it to the data frame

unformatted_genres = df['genres']

# main list of genres grouped by movies
genre_master_copy = []

# list of overall cast by individual names
genre_name_list = []

# reads through the genre string for each movie
for unformatted_genre_string in unformatted_genres:

    # creates new list for current movie
    genre_grouped = []

    # pulls genres out of the paragraph
    genre_line_list = re.findall("(?<=\'name\': )(.*?)(?=})", unformatted_genre_string)

    # converts list of genres for one movie into a string to manipulate further 
    genre_line_string = str(genre_line_list)

    # separates genre into a unique list per movie
    genre_line_split = genre_line_string.split(", ")

    # reads through each genre per movie & grabs the first one
    for genre in genre_line_split:
     
        # removes extraneous symbols from the genres
        genre = genre.replace("\"", "")
        genre = genre.replace("\'", "")
        genre = genre.replace("[", "")
        genre = genre.replace("]", "")

        genre_str = str(genre)
        break

    # adds formatted genre to list
    genre_master_copy.append(genre_str)

# assigns formatted cast to 'cast' column of DataFrame
df['genres'] = genre_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section appends the genre to the cast string. This preparation is 
# needed for text vectorization / embedding.

final_format_col = []
cast = df['cast'].to_list()
genres = df['genres'].to_list()

i = 0
count = len(cast)
for i in range(count):
    final_string = cast[i] + " " + genres[i]
    final_format_col.append(final_string)

df.insert(loc=0, column='training_col', value=final_format_col)
df.training_col = final_format_col
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section filters out data points that will be harmful for
# our model's accuracy and drops unnecessary columns.

# gets rid of all movies with 0 votes
# Number of movies before operation: 43019
# Number of movies after operation: 22651
df = df.drop(df[df.vote_count < 10].index)
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section handles the creation of the .csv files that will be the
# primary data used to train the models. 

# writes formatted DataFrame values to a new csv
pd.DataFrame.to_csv(df, "formatted_movies.csv", index="false")

# creates a csv of the actor dictionary
# w = csv.writer(open("actor_dict.csv", "w"))
# for key, val in actor_dict.items():
#     w.writerow([key, val])
# ---------------------------------------------------------------------------- #

# Preparing Training Dataset

In [16]:
np.random.seed(1)
tf.random.set_seed(1)

movie_df = df
movie_df.drop(['vote_count'], axis = 1)

data_count = movie_df.shape[0]
training_data_count = int(data_count * 0.75) + 1
test_data_count = data_count - training_data_count

y_train = np.array(movie_df['vote_average'].head(training_data_count))
x_train = np.array(movie_df['training_col'].head(training_data_count))

y_test = np.array(movie_df['vote_average'].tail(test_data_count))
x_test = np.array(movie_df[['training_col']].tail(test_data_count))

std = statistics.stdev(movie_df['vote_average'].to_list())
mean = statistics.mean(movie_df['vote_average'].to_list())
variance = statistics.variance(movie_df['vote_average'].to_list())
mode = statistics.mode(movie_df['vote_average'].to_list())

print(f"STD: {std}")
print(f"mean: {mean}")
print(f"variance: {variance}")
print(f"mode: {mode}")

# print(x_train)
print(f"x_train shape: {x_train.shape}")

STD: 0.9976277124059775
mean: 6.165617412034789
variance: 0.9952610525603838
mode: 6.5
x_train shape: (16989,)


## Text Vectorization For Actors & Genres

In [17]:
VOCAB_SIZE = 201277
encoder = preprocessing.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=6)
features = np.array(df['training_col'])
encoder.adapt(features)

# this line of code doesnt work on windows for some reason
# print(np.array(encoder.get_vocabulary()))

## Text Vectorization Sanity Check

In [18]:
print(movie_df['training_col'].to_numpy()[0])
x = encoder(movie_df['training_col'].to_numpy()[0]).numpy()
x

DakotaFanning JosePabloCantillo Horror


array([ 325, 3016,    5,    0,    0,    0], dtype=int64)

# Recurrent Neural Network

In [19]:
def build_model(hp):

    model_RNN = tf.keras.Sequential([
    encoder,
        Layer.Embedding(
            input_dim=len(encoder.get_vocabulary()),
            output_dim=hp.Int('embedding_neurons', min_value=10, max_value=110, step=50)),
        Layer.SimpleRNN(hp.Int('rnn_neurons', min_value=10, max_value=110, step=50)),
        Layer.Dense(hp.Int('dense1_neurons', min_value=10, max_value=110, step=50), activation='relu'),
        Layer.Dense(hp.Int('dense2_neurons', min_value=10, max_value=110, step=50), activation='relu'),
        Layer.Dense(hp.Int('dense3_neurons', min_value=10, max_value=110, step=50), activation='relu'),
        Layer.Dense(1)
    ])

    optimizer=hp.Choice('optimizer', values=['adam', 'RMSprop'])
    tf.keras.optimizers.Adam(hp.Choice('learning_rate', [.01, .001, .0001]))


    model_RNN.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=optimizer,
                metrics=['accuracy'])

    model_RNN.summary()
    return model_RNN




## RNN Training & Validation

In [20]:
# model_RNN.fit(
#     x_train,
#     y_train, 
#     batch_size=256,
#     epochs=10,
#     verbose=4)

# score = model_RNN.evaluate(x_test, y_test)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=300,  # how many model variations to test?
    executions_per_trial=1)  # how many trials per variation? (same model could perform differently

tuner.search_space_summary()
    
tuner.search(x=x_train,
            y=y_train,    
            verbose=2,
            epochs=1,
            batch_size=512,
            callbacks=[],
            validation_data=(x_test, y_test))

tuner.results_summary()

Invalid model 0/5


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\engine\hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_9264/3943301851.py", line 6, in build_model
    input_dim=len(encoder.get_vocabulary()),
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\text_vectorization.py", line 355, in get_vocabulary
    return self._index_lookup_layer.get_vocabulary(include_special_tokens)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\index_lookup.py", line 302, in get_vocabulary
    vocab, indices = (self._tensor_vocab_to_numpy(vocab), indices.numpy())
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\string_lookup.py", line 336, in _tensor_vocab_to

Invalid model 1/5
Invalid model 2/5


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\engine\hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_9264/3943301851.py", line 6, in build_model
    input_dim=len(encoder.get_vocabulary()),
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\text_vectorization.py", line 355, in get_vocabulary
    return self._index_lookup_layer.get_vocabulary(include_special_tokens)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\index_lookup.py", line 302, in get_vocabulary
    vocab, indices = (self._tensor_vocab_to_numpy(vocab), indices.numpy())
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\string_lookup.py", line 336, in _tensor_vocab_to

Invalid model 3/5
Invalid model 4/5
Invalid model 5/5


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_tuner\engine\hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_9264/3943301851.py", line 6, in build_model
    input_dim=len(encoder.get_vocabulary()),
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\text_vectorization.py", line 355, in get_vocabulary
    return self._index_lookup_layer.get_vocabulary(include_special_tokens)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\index_lookup.py", line 302, in get_vocabulary
    vocab, indices = (self._tensor_vocab_to_numpy(vocab), indices.numpy())
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\preprocessing\string_lookup.py", line 336, in _tensor_vocab_to

RuntimeError: Too many failed attempts to build model.

### Validating Training Data

In [ ]:
# print(df.loc[df['title'] == 'Sully']['vote_average'])

# sully_input = 'TomHanks AaronEckhart LauraLinney AnnaGunn AutumnReeser Drama'
# validation_prediction = model_RNN.predict(np.array([sully_input]))
# print(validation_prediction[0])

### Validating Test Data

In [ ]:
# def print_test_prediction(index):
#     print(f"Prediction for index: {index}")
    
#     print(f"input ---> {x_test[index]}")
#     print(f"actual output ---> {y_test[index]}")
#     print(f"predicted output ---> {model_RNN.predict(x_test[index])}\n")
        
# print_test_prediction(0)
# print_test_prediction(2)
# print_test_prediction(3)
# print_test_prediction(4)
# print_test_prediction(5)
# print_test_prediction(6)